# Introduction

## Foreword

This is the final notebook that tests our best model on the testing data set.

## Problem Statement

The rapid evolution of generative artificial intelligence (GPAI, LLMs) social media has rapidly increased the public’s access to powerful, deceptive tools. One such concern is the increasing prevalence of deepfake images, which pose a significant threat to public trust and undermines the epistemic integrity of visual media. (Source).

These manipulated images can be utilized to spread false information, manipulate public opinion, and polarize communities, which can have serious consequences for both social and political discourse. In this project, we aim to develop a machine learning model that can detect differences between deepfakes and real images to combat the spread of manipulated visual media and protect the integrity of social discourse.

## Imports, Global Variables, and Helper Functions

In [1]:
import os
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Resizing
import matplotlib.pyplot as plt

In [2]:
!unzip effnet_deepfake_model.zip

## Data Loading

In [12]:
fake_image_directory = './example_images/Fake/'
real_image_directory = './example_images/Real/'

fake_images = [fake_image_directory + x for x in os.listdir(fake_image_directory)]
real_images = [real_image_directory + x for x in os.listdir(real_image_directory)]

images = fake_images + real_images

In [ ]:
def get_random_image(image_list):
    return np.random.choice(images, 1)[0]

sample_image = get_random_image(images)
image_array = img_to_array(load_img(sample_image))

# Modeling

## Loading

In [22]:
model_path = './dffnetv2b0.json'
weights_path = './dffnetv2b0_weights.h5'

model_file = open(model_path, 'r')
loaded_model_file = model_file.read()
model_file.close()

loaded_model = tf.keras.models.model_from_json(loaded_model_file)
loaded_model.load_weights(weights_path)

## Testing

## Graphing

EDA graphs suggestions
best model -> line plot (loss/accruacy history)
best model -> roc curve (auc)
all models -> bar graph (precision scores)
all models -> bar graph (recall scores)
4:26
metrics
positive = deepfake
false negative = it is a deepfake, but we dont detect it
recall score helps track this metric
4:27
business sellpoints
we combat false news and protect identities
our maybe has n accuracy & recall scores, so its really good
models are pre-trained, out of box rdy to use.
models are also modular -> future training is very easy